# Package initiation

In [2]:
%load_ext autoreload
%autoreload 2

import os
import re
import shutil
import random
import pprint
import itertools
import functools
import collections

import pysam
import pyranges as pr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_packages')

In [3]:
from handygenome import common
from handygenome.common import ChromDict, Vcfspec, Interval
from handygenome.variantplus.breakends import Breakends
from handygenome.variantplus.variantplus import VariantPlus, VariantPlusList
from handygenome.variantplus.vcfplus import VcfPlus
from handygenome.igvhandle import IGVHandle

from handygenome.variantplus import vpfilter as libvpfilter
from handygenome.vcfeditor.initvcf import create_header

In [4]:
FASTA_PATH_HG19 = "/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta"
FASTA_PATH_HG38 = "/home/users/data/01_reference/human_g1k_v38/Homo_sapiens_assembly38.fasta"

FASTA_HG19 = pysam.FastaFile(FASTA_PATH_HG19)
FASTA_HG38 = pysam.FastaFile(FASTA_PATH_HG38)
CHROMDICT_HG19 = ChromDict(fasta_path=FASTA_PATH_HG19)
CHROMDICT_HG38 = ChromDict(fasta_path=FASTA_PATH_HG38)

# Basic setups

In [5]:
SAMPLEIDS = [
    '14', '6', '87', 'F13', 'F2', 'F33', 'F37', 'FF1', 'FF104', 'FF115', 'FF13', 'FF18', 'FF20', 'FF21', 'FF23', 'FF24', 'FF27', 
    'FF3', 'FF31', 'FF34', 'FF37', 'FF39', 'FF4', 'FF43', 'FF53', 'FF56', 'FF57', 'FF58', 'FF6', 'FF62', 'FF67', 'FF71', 'FF76', 
    'FF77', 'FF78', 'FF79', 'FF80', 'FF85', 
    #'IO04',  # swapped sample
    'IO05', 'IO06', 'IO07', 'IO08', 'IO09', 'IO12', 'IO13', 'IO15', 'IO16', 'IO17', 
    'IO18', 'IO22', 'IO23', 'IO24', 'IO25', 'SC126', 'SC134', 'SC31', 'SC81', 'SC88', 'SC97', 'SC98', 
]
SAMPLEIDS_LU = [f'LU-{x}' for x in SAMPLEIDS]
SAMPLEIDS_LU_PANEL = [f'{x}_panel' for x in SAMPLEIDS_LU]
SAMPLEIDS_LU_TUMOR = [f'{x}_tumor' for x in SAMPLEIDS_LU]

BAM_TOPDIR = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/'
PANEL_REGION_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/exon_intron_targetregions_merged.bed'
PANEL_REGION_EXONS_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/SNUH_FIRST_Lung_Cancer_V5_exon_Regions_tracknamechanged.bed'
PANEL_REGION_INTRONS_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/SNUH_FIRST_Lung_Cancer_V5_intron_Regions_tracknamechanged.bed'
WGS_TMB_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/wgs_TMB_previously_found_210312.txt'

In [6]:
BAM_PATHS = {'normal': dict(), 'tumor': dict(), 'panel': dict()}
for top, dirs, files in os.walk(BAM_TOPDIR):
    for f in files:
        for sampleid in SAMPLEIDS_LU:
            for sampletype in ('tumor', 'normal', 'panel'):
                if f == f'{sampleid}.{sampletype}.bam':
                    BAM_PATHS[sampletype][sampleid] = os.path.join(top, f)

In [7]:
BAMS = {sampletype: {sampleid: pysam.AlignmentFile(bampath) 
                     for sampleid, bampath in bampathdic.items()}
        for sampletype, bampathdic in BAM_PATHS.items()}

In [8]:
PANEL_REGION_GR = pr.read_bed(PANEL_REGION_PATH)

In [9]:
WHOLE_GENOME_LENGTH = 3_095_677_412
PANEL_REGION_LENGTH = 1_890_272

In [10]:
igv = IGVHandle(60387)

In [11]:
def load_bams(sampleid):
    igv.load([BAM_PATHS['normal'][sampleid], BAM_PATHS['tumor'][sampleid], BAM_PATHS['panel'][sampleid]])

In [12]:
def show_vp_igv(vp, sampleid):
    igv.cmd('new')
    igv.load([PANEL_REGION_PATH, PANEL_REGION_EXONS_PATH, PANEL_REGION_INTRONS_PATH])
    load_bams(sampleid)

    igv.goto([vp.vcfspec], width=200)
    igv.cmd('viewaspairs')
    igv.cmd('sort base')

# Code development

### get_active_range

In [131]:
import collections

from handygenome.readplus import readhandler


def call_pileup(bam, chrom, start0, end0):
    pup = bam.pileup(chrom, start0, end0, 
                     truncate=True, max_depth=10**6, stepper='all', 
                     ignore_overlaps=False, ignore_orphans=False, 
                     min_base_quality=0, min_mapping_quality=0, 
                     adjust_capq_threshold=0, compute_baq=False, 
                     redo_baq=False)    

    return pup

    
def get_max_vaf_pysampileup(pupcol):
    counts = collections.Counter(pupcol)
    max_vaf = max(counts.values()) / sum(counts.values())
    return max_vaf


def get_is_active_pupcol(pupcol, threshold):
    max_vaf = get_max_vaf_pysampileup(
        x.upper() for x in
        pupcol.get_query_sequences(mark_matches=False, mark_ends=False, add_indels=True)
    )
    is_active = max_vaf <= threshold
    return pupcol.reference_pos, is_active


def get_active_info_from_pileup(pup, threshold):
    return [get_is_active_pupcol(pupcol, threshold) for pupcol in pup]


def check_active_col_pysampileup_rightward(bam, chrom, pos0_init, threshold, add_pileup_by):
    start0 = pos0_init
    end0 = start0 + add_pileup_by
    while True:
        pup = call_pileup(bam, chrom, start0, end0)
        for pupcol in pup:
            pos0, is_active = get_is_active_pupcol(pupcol, threshold)
            yield pos0, is_active

        start0 = end0
        end0 = start0 + add_pileup_by
        continue

        
def check_active_col_pysampileup_leftward(bam, chrom, pos0_init, threshold, add_pileup_by):
    end0 = pos0_init + 1
    start0 = end0 - add_pileup_by
    while True:
        pup = call_pileup(bam, chrom, start0, end0)
        active_info_buffer = get_active_info_from_pileup(pup, threshold)
        for pos0, is_active in reversed(active_info_buffer):
            yield pos0, is_active

        end0 = start0
        start0 = end0 - add_pileup_by
        continue

            
def get_active_range_pysampileup(bam, chrom, start0, end0, threshold=0.9, inactive_padding=5, add_pileup_by=10):
        
#     readlist = list(bam.fetch(chrom, start0, end0))
#     pileup_df = get_pileup(readlist, as_df=True)
    pup = call_pileup(bam, chrom, start0, end0)
    naive_active_info = get_active_info_from_pileup(pup, threshold)        
    naive_active_positions = [pos0 for (pos0, is_active) in naive_active_info if is_active]    
    if len(naive_active_positions) == 0:
        return
    
    naive_active_positions_min = min(naive_active_positions)
    naive_active_positions_max = max(naive_active_positions)
    
#     idx_min = naive_active_info.index((naive_active_positions_min, True))
#     idx_max = naive_active_info.index((naive_active_positions_max, True))
#     naive_active_info_filtered = naive_active_info[idx_min:(idx_min + 1)]
    
    active_info_rightward = list()
    active_info_leftward = list()
    
    # rightward
    current_pos0 = naive_active_positions_max + 1
    for pos0, is_active in check_active_col_pysampileup_rightward(
        bam, chrom, current_pos0, threshold=threshold, add_pileup_by=add_pileup_by
    ):
        active_info_rightward.append((pos0, is_active))
        if len(active_info_rightward) >= inactive_padding:
            if not any(x[1] for x in active_info_rightward[-inactive_padding:]):
                break

    # leftward
    current_pos0 = naive_active_positions_min - 1
    for pos0, is_active in check_active_col_pysampileup_leftward(
        bam, chrom, current_pos0, threshold=threshold, add_pileup_by=add_pileup_by
    ):
        active_info_leftward.append((pos0, is_active))
        if len(active_info_leftward) >= inactive_padding:
            if not any(x[1] for x in active_info_leftward[-inactive_padding:]):
                break
                
#     active_info = active_info_leftward[::-1] + naive_active_info_filtered + active_info_rightward
#     positions = [pos0 for (pos0, is_active) in active_info]    
    active_range = range(active_info_leftward[-1][0], active_info_rightward[-1][0] + 1)
    
    return active_range


###############################

def get_max_vaf_mypileup(col):
    counts = collections.Counter(x for x in col if x is not None)
    max_vaf = max(counts.values()) / sum(counts.values())    
    return max_vaf

def get_iterator(pileup, as_array, start0, end0):
    if as_array:
        rng = range(start0, end0)
        return ((rng[idx], pileup[:, idx]) for idx in range(pileup.shape[1]))
    else:
        return pileup.items()

    
def get_active_info_mypileup(bam, chrom, start0, end0, threshold, as_array):
    pileup = readhandler.get_pileup(bam, chrom, start0, end0, as_array=as_array, truncate=True)
    result = list()

    for pos0, col in get_iterator(pileup, as_array, start0, end0):
        is_active = (get_max_vaf_mypileup(col) <= threshold)
        result.append((pos0, is_active))
    return result


def check_active_col_mypileup_rightward(bam, chrom, pos0_init, threshold, add_pileup_by, as_array):
    start0 = pos0_init
    end0 = start0 + add_pileup_by
    while True:
        pileup = readhandler.get_pileup(bam, chrom, start0, end0, as_array=as_array, truncate=True)
        for pos0, col in get_iterator(pileup, as_array, start0, end0):
            is_active = (get_max_vaf_mypileup(col) <= threshold)
            yield pos0, is_active

        start0 = end0
        end0 = start0 + add_pileup_by
        continue
        
        
def check_active_col_mypileup_leftward(bam, chrom, pos0_init, threshold, add_pileup_by, as_array):
    end0 = pos0_init + 1
    start0 = end0 - add_pileup_by
    while True:
        pileup = readhandler.get_pileup(bam, chrom, start0, end0, as_array=as_array, truncate=True)
        for pos0, col in reversed(tuple(get_iterator(pileup, as_array, start0, end0))):
            is_active = (get_max_vaf_mypileup(col) <= threshold)
            yield pos0, is_active

        end0 = start0
        start0 = end0 - add_pileup_by
        continue    
    
###
    

def get_active_range_mypileup(bam, chrom, start0, end0, threshold=0.9, inactive_padding=5, add_pileup_by=10, as_array=False):
    naive_active_info = get_active_info_mypileup(bam, chrom, start0, end0, threshold, as_array)
    naive_active_positions = [pos0 for (pos0, is_active) in naive_active_info if is_active]    
    if len(naive_active_positions) == 0:
        return
    
    naive_active_positions_min = min(naive_active_positions)
    naive_active_positions_max = max(naive_active_positions)
        
    active_info_rightward = list()
    active_info_leftward = list()
    
    # rightward
    current_pos0 = naive_active_positions_max + 1
#     n = 0
    for pos0, is_active in check_active_col_mypileup_rightward(
        bam, chrom, current_pos0, threshold=threshold, add_pileup_by=add_pileup_by, as_array=as_array,
    ):
#         n += 1
#         if n == 500:
#             print('overflow')
#             break
        active_info_rightward.append((pos0, is_active))
        if len(active_info_rightward) >= inactive_padding:
            if not any(x[1] for x in active_info_rightward[-inactive_padding:]):
                break

    # leftward
    current_pos0 = naive_active_positions_min - 1
#     n = 0
    for pos0, is_active in check_active_col_mypileup_leftward(
        bam, chrom, current_pos0, threshold=threshold, add_pileup_by=add_pileup_by, as_array=as_array,
    ):
#         n += 1
#         if n == 500:
#             print('overflow')
#             break
        active_info_leftward.append((pos0, is_active))
        if len(active_info_leftward) >= inactive_padding:
            if not any(x[1] for x in active_info_leftward[-inactive_padding:]):
                break
                
#     active_info = active_info_leftward[::-1] + naive_active_info_filtered + active_info_rightward
#     positions = [pos0 for (pos0, is_active) in active_info]    
    active_range = range(active_info_leftward[-1][0], active_info_rightward[-1][0] + 1)
    
    return active_range

In [71]:
vcfspec = Vcfspec('7', 55_242_464, 'A', ('G',))
intv = Interval(chrom='7', start1=55_242_464, end1=55_242_480)

tbam = BAMS['tumor']['LU-FF43']
tbam_path = BAM_PATHS['tumor']['LU-FF43']
nbam = BAMS['normal']['LU-FF43']
nbam_path = BAM_PATHS['normal']['LU-FF43']

# readlist = list(BAMS['normal']['LU-FF43'].fetch(vcfspec.chrom, vcfspec.pos0, vcfspec.end0))
# readlist = list(BAMS['tumor']['LU-FF43'].fetch(vcfspec.chrom, vcfspec.pos0, vcfspec.end0))
# readlist = list(tbam.fetch(intv.chrom, intv.start0, intv.end0))

In [19]:
igv.load([tbam_path, nbam_path])
igv.goto([vcfspec], width=100)

OK
OK
OK


In [156]:
inactive_padding = 20
threshold = 0.9

In [157]:
%%timeit
active_range = get_active_range_pysampileup(bam, intv.chrom, intv.start0, intv.end0, inactive_padding=inactive_padding, threshold=threshold)

240 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [158]:
print(active_range)

range(55242366, 55242529)


In [159]:
%%timeit
active_range = get_active_range_mypileup(bam, intv.chrom, intv.start0, intv.end0, inactive_padding=inactive_padding, as_array=True, threshold=threshold)

43.6 ms ± 773 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [160]:
print(active_range)

range(55242366, 55242529)


In [161]:
%%timeit
active_range = get_active_range_mypileup(bam, intv.chrom, intv.start0, intv.end0, inactive_padding=inactive_padding, as_array=False, threshold=threshold)

53.3 ms ± 640 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [162]:
print(active_range)

range(55242366, 55242529)
